In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from collections import Counter
import string
import nltk
nltk.download('words')
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
df = pd.read_csv(r"Final Dataset.csv")
df.head()

,subject,message,label
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0
1,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0
2,risk,a colleague and i are researching the differin...,0
3,request book information,earlier this morning i was on the phone with a...,0
4,call for abstracts : optimality in syntactic t...,content - length : 4437 call for papers is the...,0


In [ ]:
df.isnull().sum()

subject    0
message    0
label      0
dtype: int64

In [ ]:
df['sub_mssg']=df['subject']+" "+df['message']
df.head()

,subject,message,label,sub_mssg
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0,job posting - apple-iss research center conten...
1,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0,query : letter frequencies for text identifica...
2,risk,a colleague and i are researching the differin...,0,risk a colleague and i are researching the dif...
3,request book information,earlier this morning i was on the phone with a...,0,request book information earlier this morning ...
4,call for abstracts : optimality in syntactic t...,content - length : 4437 call for papers is the...,0,call for abstracts : optimality in syntactic t...


In [ ]:
df.drop('subject',axis=1,inplace=True)

In [ ]:
df.head()

,message,label,sub_mssg
0,content - length : 3386 apple-iss research cen...,0,job posting - apple-iss research center conten...
1,i am posting this inquiry for sergei atamas ( ...,0,query : letter frequencies for text identifica...
2,a colleague and i are researching the differin...,0,risk a colleague and i are researching the dif...
3,earlier this morning i was on the phone with a...,0,request book information earlier this morning ...
4,content - length : 4437 call for papers is the...,0,call for abstracts : optimality in syntactic t...


In [ ]:
df.drop('message',axis=1,inplace=True)

In [ ]:
df.head()

,label,sub_mssg
0,0,job posting - apple-iss research center conten...
1,0,query : letter frequencies for text identifica...
2,0,risk a colleague and i are researching the dif...
3,0,request book information earlier this morning ...
4,0,call for abstracts : optimality in syntactic t...


In [ ]:
import re

In [ ]:
df['sub_mssg'][5]

'm . a . in scandinavian linguistics m . a . in scandinavian linguistics at the university of tromsoe 1995-1996 ( corrected version ) the linguistics section of the institute for language and literature at the university of tromsoe  norway  offers a one year course leading up to a master of arts degree  starting the fall of 1995 . the course is designed to prepare students for research in generative syntax and phonology with special attention to the scandinavian languages . the course is open to students from all countries with a background in linguistics corresponding roughly to a b . a . with a major in linguistics  or a norwegian cand . mag . the course is equivalent to 60 e . c . t . s credits . instruction will be in english . proficiency in a scandinavian language is not required  nor is previous knowledge of scandinavian grammar . those who wish may follow a course in norwegian for foreigners during the whole year . students pay no tuition  only a nominal registration fee . a li

In [ ]:
#REPLACING NUMBERS
df['sub_mssg']=df['sub_mssg'].str.replace(r'\d+(\.\d+)?', 'numbers')
df['sub_mssg'][5]

'm . a . in scandinavian linguistics m . a . in scandinavian linguistics at the university of tromsoe numbers-numbers ( corrected version ) the linguistics section of the institute for language and literature at the university of tromsoe  norway  offers a one year course leading up to a master of arts degree  starting the fall of numbers . the course is designed to prepare students for research in generative syntax and phonology with special attention to the scandinavian languages . the course is open to students from all countries with a background in linguistics corresponding roughly to a b . a . with a major in linguistics  or a norwegian cand . mag . the course is equivalent to numbers e . c . t . s credits . instruction will be in english . proficiency in a scandinavian language is not required  nor is previous knowledge of scandinavian grammar . those who wish may follow a course in norwegian for foreigners during the whole year . students pay no tuition  only a nominal registrat

In [ ]:
#CONVRTING EVERYTHING TO LOWERCASE
df['sub_mssg']=df['sub_mssg'].str.lower()
#REPLACING NEXT LINES BY 'WHITE SPACE'
df['sub_mssg']=df['sub_mssg'].str.replace(r'\n'," ") 
# REPLACING EMAIL IDs BY 'MAILID'
df['sub_mssg']=df['sub_mssg'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','MailID')
# REPLACING URLs  BY 'Links'
df['sub_mssg']=df['sub_mssg'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','Links')
# REPLACING CURRENCY SIGNS BY 'MONEY'
df['sub_mssg']=df['sub_mssg'].str.replace(r'£|\$', 'Money')
# REPLACING LARGE WHITE SPACE BY SINGLE WHITE SPACE
df['sub_mssg']=df['sub_mssg'].str.replace(r'\s+', ' ')

# REPLACING LEADING AND TRAILING WHITE SPACE BY SINGLE WHITE SPACE
df['sub_mssg']=df['sub_mssg'].str.replace(r'^\s+|\s+?$', '')
#REPLACING CONTACT NUMBERS
df['sub_mssg']=df['sub_mssg'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','contact number')
#REPLACING SPECIAL CHARACTERS  BY WHITE SPACE 
df['sub_mssg']=df['sub_mssg'].str.replace(r"[^a-zA-Z0-9]+", " ")

In [ ]:
df['sub_mssg'][0]

'job posting apple iss research center content length numbers apple iss research center a us Money numbers million joint venture between apple computer inc and the institute of systems science of the national university of singapore located in singapore is looking for a senior speech scientist the successful candidate will have research expertise in computational linguistics including natural language processing and english and chinese statistical language modeling knowledge of state of the art corpus based n gram language models cache language models and part of speech language models are required a text to speech project leader the successful candidate will have research expertise expertise in two or more of the following areas computational linguistics including natural language parsing lexical database design and statistical language modeling text tokenization and normalization prosodic analysis substantial knowledge of the phonology syntax and semantics of chinese is required know

In [ ]:
# import spacy
# nlp = spacy.load("en_core_websm")

# doc = nlp("Mando talked for 3 hours although talking isn't his thing")
# doc = nlp("eating eats eat ate adjustable rafting ability meeting better")
# for token in doc:
#     print(token, " | ", token.lemma)

In [ ]:
len(df['sub_mssg'])

4196

In [ ]:
len(df['sub_mssg'][1])

1446

In [ ]:
df['sub_mssg'][1]

'query letter frequencies for text identification i am posting this inquiry for sergei atamas satamas umabnet ab umd edu a research associate at the university of maryland at baltimore his field is molecular biology and his work involves comparing dna strings using various algorithms i do n t understand the details well enough to pass them along at any rate one such algorithm relies upon frequencies with which the letters g a t and c occur in the dna strings he would like to explore the analogous use of letter sound frequencies in natural language texts hence this posting specifically sergei wonders if any linguist subscribers could help steer him to recent literature concerning text identification based on letter frequencies any suggestions could be sent directly to him at the above address or to me and i ll pass them along he would also be interested in collaborative work if this research connects with the work of any linguists or text processing specialists he observes that very oft

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# sm = ""
# temp = str(df['sub_mssg'][1])
# doc = nlp(temp)
# for token in doc:
#   sm = sm + token.lemma_ + " "


In [ ]:
words = set(nltk.corpus.words.words())

In [ ]:
# sent = "Io andiamo to the beach with my amico."
# temp = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())
# temp

In [ ]:
sm = ""
uniqueWords = {"call"}
for i in range(len(df['sub_mssg'])):
  sm = ""
  temp = str(df['sub_mssg'][i])
  temp = " ".join(w for w in nltk.wordpunct_tokenize(temp) if w.lower() in words or not w.isalpha())
  doc = nlp(temp)
  for token in doc:
    sm = sm + token.lemma_ + " "
    uniqueWords.add(token.lemma_)

  df['sub_mssg'][i] = sm


In [ ]:
# sm

In [ ]:
# len(sm)

In [ ]:
# if sm != df['sub_mssg'][1]:
#   print("ho gayaaaa")
# else:
#   print("lg gaye")

In [ ]:

# sm = ""

# for i in range(len(df['sub_mssg'])):
#   sm= ""
#   temp = str(df['sub_mssg'][i])
#   doc = nlp(temp)
#   for token in doc:
#     sm = sm + token.lemma_ + " "
  
#   df['sub_mssg'][i] = sm
# sm = ""
# temp = str(df['sub_mssg'][0])
# doc = nlp(temp)
# for token in doc:
#   sm = sm + token.lemma_ + " "
# len(sm)

In [ ]:
len(df['sub_mssg'][7])

897

In [ ]:
len(uniqueWords)

21768

In [ ]:
len(uniqueWords)

21768

In [ ]:
df.head()

,label,sub_mssg
0,0,job post apple research center content length ...
1,0,query letter for text identification I be post...
2,0,risk a colleague and I be the of risk by our h...
3,0,request book information this morning I be on ...
4,0,call for in syntactic theory content length ca...


In [ ]:
df.to_csv("Updated.csv",index=False)

In [ ]:
# file = open('geek.txt', 'r')
# # This will print every line one by one in the file
# for each in file:
#     print (each)

In [ ]:
file = open("unique words.txt","w")

In [ ]:
# uniqueWords

In [ ]:
for i in uniqueWords:
  file.write(i+" ")

In [ ]:
if '' in uniqueWords:
  print("YEs")
else:
  print("no")

no
